In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import requests
from creds import google_api_key
import os
import pandas as pd

In [2]:
partysummary = os.path.join('output','PartySummary.csv')

In [3]:
df = pd.read_csv(partysummary)

In [4]:
df.head()

,Party,Total Contestants,Won,Second,Third,Party Votes,Valid Votes %,Valid Votes % (Contested),AC Leads,AC Second,AC Third
0,Bharatiya Janta Party,428,282,54,78,171657549,31.3%,40%,1883,550,601
1,Indian National Congress,464,44,224,66,106938242,19.5%,22.6%,550,1748,531
2,All India Anna Dravida Munnetra Kazhagam,40,37,1,2,18115825,3.3%,44.4%,222,16,24
3,All India Trinamool Congress,131,34,3,7,21259681,3.9%,15.4%,215,55,90
4,Biju Janata Dal,21,20,1,0,9491497,1.7%,44.8%,116,26,5


In [5]:
urls = []
for x in range(0,9):
    urls.append('https://www.google.com/search?source=lnms&tbm=nws&q='+ df['Party'][x].replace(' ','+'))

In [6]:
all_list = []

for url in urls:
    webpage = requests.get(url)
    soup = bs(webpage.text, 'lxml')
    results = soup.find_all('h3', class_ = 'r')
    
    news_headlines = []
    for x in range(len(results)):
        news_headlines.append((results[x].text))
    all_list.append(news_headlines)

In [7]:
mydict = {}

for x in range(len(all_list)):
    mydict[df['Party'][x]] = all_list[x]

In [8]:
mydict

{'Bharatiya Janta Party': ["India's Liberal Expats Are Modi's Biggest Fans",
  'BJP threatens Karnataka stir over drought',
  'Looks difficult for Bharatiya Janata Party to reach 280-mark: Shiv ...',
  "India's BJP releases manifesto before elections",
  'Daler Mehndi joins Bharatiya Janata Party',
  'Regional parties will play key role to form govt: Kerala CM Pinarayi ...',
  'Disgruntled MP Udit Raj quits Bharatiya Janata Party, joins Congress',
  'BJP files new batch of complaints to Election Commission',
  'Lok Sabha Elections 2019: Bollywood Actor Sunny Deol Joins ...',
  '9 attacks in 5 years: Arvind Kejriwal blames Bharatiya Janata Party'],
 'Indian National Congress': ["Congress writes to EC; demands action over PM Modi's 'Rajiv ...",
  'No, this video does not show an Indian National Congress ...',
  'As it happened | General election, Phase 5: Overall turnout stands at ...',
  'Star wars in Indian National Congress',
  'Kangana Ranaut attacks Congress on Mumbai poll day: We w

## Sentiment Analysis using Google Language AI/Machine Learning 

In [9]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'creds.json'

In [10]:
sentiment_list = []
newdict = {}

In [16]:
client = language.LanguageServiceClient()

for key, value in mydict.items():
    for headlines in value:
        document = types.Document(content=headlines, type=enums.Document.Type.PLAIN_TEXT)
        sentiment = client.analyze_sentiment(document=document).document_sentiment.score
        sentiment_list.append(sentiment)
    newdict[key] = sentiment_list
    sentiment_list = []

In [18]:
newsdf = pd.DataFrame.from_dict(newdict)

In [24]:
newsdf.describe()

,Bharatiya Janta Party,Indian National Congress,All India Anna Dravida Munnetra Kazhagam,All India Trinamool Congress,Biju Janata Dal,Shiv Sena,Telugu Desam,Telangana Rashtra Samithi,Communist Party Of India (MARXIST)
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000
mean,-0.010000,0.020000,0.050000,-0.160000,-0.040000,-0.260000,0.030000,0.130000,-0.06000
std,0.417532,0.278089,0.287711,0.492612,0.298887,0.452647,0.226323,0.235938,0.33731
min,-0.800000,-0.300000,-0.400000,-0.800000,-0.600000,-0.900000,-0.400000,-0.100000,-0.90000
25%,-0.225000,-0.150000,0.000000,-0.625000,-0.225000,-0.725000,0.000000,0.000000,-0.10000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,-0.100000,0.050000,0.000000,0.00000
75%,0.250000,0.000000,0.150000,0.075000,0.200000,0.075000,0.200000,0.175000,0.07500
max,0.700000,0.700000,0.700000,0.800000,0.300000,0.300000,0.300000,0.600000,0.30000


In [25]:
newsdf.describe().to_csv(os.path.join('output','SentimentScores.csv'))